# Introducción

**Alumna**: Abril Nazarena Rodriguez. Legajo: R-4582/9

**Asignatura**: Procesamiento del Lenguaje Natural.

---





El presente proyecto implica el desarrollo de un chatbot especializado que empleará la técnica RAG (Retrieve, Answer, Generate) para interactuar con los usuarios y proporcionar información detallada sobre el mundo de los superhéroes. Este chatbot se basará en tres fuentes de datos fundamentales: datos numéricos tabulares, una base de datos de grafos y una base de datos vectorial.

Como el núcleo del chatbot estará centrado en el universo de los superhéroes, ofrecerá información tanto sobre la historia de los cómics como sobre las características distintivas de los propios superhéroes. Estos datos estarán alojados en una base de datos vectorial de ChromaDB, previamente alimentada con información extraída de variados archivos PDF. Antes de integrarlos, se llevará a cabo un proceso que abarcará desde la limpieza hasta la segmentación de los datos, culminando con la aplicación de técnicas de embedding para lograr una representación eficiente y estructurada en la base de datos.

Para ampliar aún más la base de conocimientos del chatbot, se integrará una conexión con una base de datos de grafos en Wikidata. Esto permitirá al usuario obtener información precisa sobre un superhéroe específico.

Finalmente, la base de datos tabular incluirá información relacionada con los lanzamientos cinematográficos más destacados de superhéroes. Aspectos como las ventas, fechas de lanzamiento, duración de las películas y otros datos relevantes le permitirán al usuario realizar consultas acerca de dichas adaptaciones cinematográficas.

#Librerías utilizadas

Se instalarán e importarán librerías como langchain, chromaDB, sentence-transformers, entre otras que serán necesarias para la ejecución del chatbot.

In [ ]:
!pip install langchain
!pip install chromadb
!pip install wikidataintegrator
!pip install sparqlwrapper
!pip install sentence-transformers
!pip install PyMuPDF
!pip install nltk
!pip install PyMuPDF
!python -m spacy download es_core_news_sm

2024-01-27 00:17:27.401636: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-27 00:17:27.401715: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-27 00:17:27.403635: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-27 00:17:27.415206: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-27 00:17:28.930768: W tensorflow/compiler/tf2

In [ ]:
# Extracción y manipulación de texto, BBDD vectorial
import fitz  # PyMuPDF
import unicodedata
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
import pandas as pd
import chromadb

# BBDD de Grafos
from wikidataintegrator import wdi_core, wdi_login
import requests
import json

# Chatbot
from jinja2 import Template
from typing import Any, Dict, List

# Ejercicio 1

Para nutrir de conocimientos al chatbot, utilicé las siguientes fuentes:

- Archivos pdf: extraidos de las siguientes páginas.

*-* https://riuma.uma.es/xmlui/bitstream/handle/10630/6994/Bajo%20la%20piel%20de%20los%20superheroes.pdf?sequence=6&isAllowed=y

*-* https://www.bibliotecaspublicas.es/dam/jcr:bbd9f090-6390-4801-89d6-8ca9103772dd/comic_superheroes.pdf

*-* https://www.redalyc.org/journal/5644/564462902004/564462902004.pdf

*-* https://www.juntadeandalucia.es/averroes/centros-tic/11000289/helvia/aula/archivos/repositorio/0/126/teoria_2_ESO_-_9_COMIC.pdf

*-* https://editorial.urosario.edu.co/pageflip/acceso-abierto/los-superheroes.pdf

*-* https://historietasargentinas.files.wordpress.com/2017/06/las-vidas-de-los-superhc3a9roes.pdf

- Base de datos de grafos: utilicé una base de datos de grafos online, Wikidata.

- Datos tabulares: archivo csv extraído de Openintro:
https://www.openintro.org/data/index.php?data=mcu_films  


## Base de datos vectorial


En esta sección, se llevará a cabo el desarrollo de la base de datos vectorial de ChromaDB, la cual contendrá información extraída de diversos documentos PDF que exploran diversos aspectos de la literatura relacionada con superhéroes. Estos documentos abordarán temas como el comportamiento de los superhéroes, su historia, su presencia en cómics, entre otros.

El proceso se iniciará mediante la carga de los documentos PDF en variables específicas, seguido por la extracción sistemática de la información relevante. Para garantizar la calidad y coherencia de los datos, se implementarán técnicas de limpieza adaptadas a las particularidades de la información contenida en dichos documentos.

Posteriormente, se llevará a cabo la segmentación de la información en fragmentos más manejables, conocidos como "chunks". Esta estrategia facilitará la posterior manipulación y consulta de datos en la base de datos vectorial.

El siguiente paso consistirá en la generación de embeddings, una representación numérica de la información, que permitirá una organización eficiente y una recuperación rápida de datos. Estos embeddings se almacenarán en la base de datos vectorial de ChromaDB, asegurando así un acceso eficaz y optimizado a la información sobre la literatura de superhéroes.

In [ ]:
pdf_1 = 'Bajo la piel de los superheroes.pdf'
pdf_2 = 'comic_superheroes.pdf'
pdf_3 = 'mujeres_superheroes.pdf'
pdf_4 = 'teoria_2_ESO_-_9_COMIC.pdf'
pdf_5 = 'los-superheroes.pdf'
pdf_6 = 'las-vidas-de-los-superhc3a9roes.pdf'

En la celda de código anterior, he definido seis variables, cada una asociada a un archivo PDF distinto.

A continuación, he creado la función 'extraer_texto', diseñada para recibir un archivo PDF como entrada y devolver el texto extraído. De manera opcional, se puede especificar el rango de páginas que se desea extraer.

Dado que algunos de los archivos contienen información no relevante al inicio o al final, opté por invocar la función de extracción de texto de manera individual seis veces, una para cada archivo, con el fin de obtener segmentos específicos de texto.

In [ ]:
def extraer_texto(pdf_path, start_page=0, end_page=None):
    '''Recibe un archivo pdf y devuelve su texto extraido. Opcionalmente,
    se puede indicar una página de inicio y una página de fin.'''
    text = ''
    with fitz.open(pdf_path) as pdf_document:
        end_page = end_page if end_page is not None else pdf_document.page_count
        for page_num in range(start_page, end_page):
            page = pdf_document[page_num]
            text += page.get_text("text")
    # Eliminar guiones y nuevas lineas
    text = text.replace('-', '').replace('\n', '')
    return text

# Extraer texto de los PDFs
texto_1 = extraer_texto(pdf_1, end_page = 24) # 'Bajo la piel de los superheroes.pdf'
texto_2 = extraer_texto(pdf_2, start_page = 1, end_page = 12) # 'comic_superheroes.pdf'
texto_3 = extraer_texto(pdf_3, start_page = 1, end_page = 5) # 'mujeres_superheroes.pdf'
texto_4 = extraer_texto(pdf_4, end_page = 8) # 'teoria_2_ESO_-_9_COMIC.pdf'
texto_5 = extraer_texto(pdf_5, start_page = 15, end_page = 175) # texto_5 = 'los-superheroes.pdf'
texto_6 = extraer_texto(pdf_6, end_page = 15) # texto_6 = 'las-vidas-de-los-superhc3a9roes.pdf'

In [ ]:
# Ejemplo de un archivo pdf

print("\nTexto extraído de comic_superheroes.pdf: \n")
print(texto_2)


Texto extraído de comic_superheroes.pdf: 

CÓMIC DE SUPERHÉROESLa historia de los cómics de superhéroes es larga, tiene su origen en laprimera mitad del siglo XX. Teniendo en cuenta que fue una épocaconvulsa, económica y socialmente, no es de extrañar  que las aventurasprotagonizadas por personajes de moral intachable, y capaces deafrontar cualquier problema, tuvieran gran acogida.Allá por 1917 es cuando la palabra superhéroe se populariza, aunqueno con la imagen que nos viene hoy en día a la mente. Esos primerossuperhéroes se dividían en dos grupos: por un lado, estaban losvengadores enmascarados que combatían la injusticia, como El Zorro; y,por el otro, personajes caricaturescos con superpoderes como el conocidomarinero Popeye, que obtenía su gran fuerza de las espinacas.Estos dos grupos confluyeron en un tipo de héroe que oculta tanto suidentidad como sus poderes sobrenaturales para pasar desapercibidoentre la gente. En junio de 1938 National Allied Publications (que mástarde sería

Se observa como ejemplo la extracción del texto número 2. Finalmente, unifico todos los textos extraidos dentro de una única variable 'texto', el cual contiene 408.319 caracteres.

In [ ]:
texto = texto_1 + texto_2 + texto_3 + texto_4 + texto_5 + texto_6
len(texto)

408319

### Limpieza de texto

Aquí, he realizado una limpieza de la variable 'texto', previamente definida, con el objetivo de reducir el ruido, normalizarlo y mejorar la eficiencia del modelo de NLP para obtener resultados más precisos. Las técnicas de limpieza de texto que he seleccionado incluyen:

- Conversión a minúsculas para garantizar uniformidad.
- Eliminación de puntuación y acentos para simplificar el texto.
- Reemplazo de múltiples espacios con uno solo para mantener coherencia en el formato.
- Eliminación de URLs para eliminar enlaces web.
- Eliminación de stopwords para reducir palabras comunes y poco informativas.

Estas estrategias de limpieza contribuirán a optimizar la calidad de los datos de entrada y, por ende, mejorarán el rendimiento del modelo NLP.

In [ ]:
# Descargar recursos de nltk
nltk.download('stopwords')
nltk.download('punkt')

def limpiar_texto(text):
    '''Recibe un texto (str) y aplica diversas técnicas para la limpieza del mismo.'''

    # Conversión a minúsculas
    text = text.lower()

    # Eliminación de puntuación y acentos
    text = re.sub('[^\w\s.]', '', text)
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')

    def remove_accents(input_str):
        nfkd_form = unicodedata.normalize('NFKD', input_str)
        return ''.join([c for c in nfkd_form if not unicodedata.combining(c)])

    text = remove_accents(text)

    # Reemplazo de espacios múltiples por un único espacio
    text = re.sub('\s+', ' ', text)

    # Eliminación de URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

    # Eliminación de stopwords
    stop_words = set(stopwords.words('spanish'))
    word_tokens = word_tokenize(text)
    filtered_text = [word for word in word_tokens if word.casefold() not in stop_words]
    text = " ".join(filtered_text)

    return text


# Llamo a la función
texto_limpio = limpiar_texto(texto)

print("Texto limpio: \n")
print(texto_limpio)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Texto limpio: 

1 bajo piel superheroe fusion identidad imagen . jose ma alonso calero josefa cano garcia facultad bellas artes influencia superheroes imaginacion colectiva basa encarnacion esperanzas suenos deseos . metaforas realidad social politica tambien llegado ser referentes terminos sexualidad culto cuerpo siendo representativos cuerpo humano forma hiperbolica . apariencia redefine adapta segun canones populares belleza llegar ser siempre superlativo . poder transformacion punto mas importante metafora encarna . metamorfosis da capacidad rehacerse volver hacerse vez ave fenix dandonos posibilidad fantasear escaparnos banal ordinario cotidiano . capitulo trataremos aspectos iconicos figura superheroe buscaremos dar respuesta simbolo metafora visual aspectos psicologicos ideologicos politicos personales personaje ven demas capitulo libro . elementos esteticos destacar mundo superheroes irian iconicos simbolicos cuestiones abarcan estilo grafico artistico audiovisual llegando aspe

### Chunks

Para dividir el texto en fragmentos más pequeños, empleé el método RecursiveCharacterTextSplitter de Langchain. En este proceso, opté por utilizar el punto como delimitador y establecí un tamaño máximo de chunk de 500 caracteres.

Esta configuración me permitió segmentar el contenido en fragmentos más manejables, cada uno compuesto por varias oraciones.

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(separators = ["."], chunk_size=500)

# Aplico la división a los textos de los PDFs
texts = text_splitter.split_text(texto_limpio)

for txt in texts:
    print(f'{len(txt)}: {txt}')

446: 1 bajo piel superheroe fusion identidad imagen . jose ma alonso calero josefa cano garcia facultad bellas artes influencia superheroes imaginacion colectiva basa encarnacion esperanzas suenos deseos . metaforas realidad social politica tambien llegado ser referentes terminos sexualidad culto cuerpo siendo representativos cuerpo humano forma hiperbolica . apariencia redefine adapta segun canones populares belleza llegar ser siempre superlativo
484: . apariencia redefine adapta segun canones populares belleza llegar ser siempre superlativo . poder transformacion punto mas importante metafora encarna . metamorfosis da capacidad rehacerse volver hacerse vez ave fenix dandonos posibilidad fantasear escaparnos banal ordinario cotidiano . capitulo trataremos aspectos iconicos figura superheroe buscaremos dar respuesta simbolo metafora visual aspectos psicologicos ideologicos politicos personales personaje ven demas capitulo libro
497: . capitulo trataremos aspectos iconicos figura superh

Se verifica que ningún chunk sobrepasa el límite de 500 caracteres establecido.

No obstante, dado que empleé el punto como delimitador, se observa la presencia de un punto al inicio de cada chunk. Por este motivo, en la siguiente celda de código, he creado una función destinada a limpiar los chunks y eliminar los puntos iniciales.

In [ ]:
def clean_chunks(chunks):
  '''Recibe chunks y elimina el punto al inicio de cada uno'''
  cleaned_chunks = []
  for chunk in chunks:
      # Eliminar puntos al principio
      chunk = chunk.lstrip('.')
      cleaned_chunks.append(chunk)

  return cleaned_chunks

chunks_limpios = clean_chunks(texts)

for txt in chunks_limpios:
  #Imprimo la longitud de la cadena, y luego el trozo de texto (chunk)
  print(f'{len(txt)}: {txt}')

446: 1 bajo piel superheroe fusion identidad imagen . jose ma alonso calero josefa cano garcia facultad bellas artes influencia superheroes imaginacion colectiva basa encarnacion esperanzas suenos deseos . metaforas realidad social politica tambien llegado ser referentes terminos sexualidad culto cuerpo siendo representativos cuerpo humano forma hiperbolica . apariencia redefine adapta segun canones populares belleza llegar ser siempre superlativo
483:  apariencia redefine adapta segun canones populares belleza llegar ser siempre superlativo . poder transformacion punto mas importante metafora encarna . metamorfosis da capacidad rehacerse volver hacerse vez ave fenix dandonos posibilidad fantasear escaparnos banal ordinario cotidiano . capitulo trataremos aspectos iconicos figura superheroe buscaremos dar respuesta simbolo metafora visual aspectos psicologicos ideologicos politicos personales personaje ven demas capitulo libro
496:  capitulo trataremos aspectos iconicos figura superher

### Embeddings e incersión a ChromaDB

Una vez obtenidos los chunks con el texto limpio, el siguiente paso es la inserción de estos fragmentos en la base de datos vectorial. Sin embargo, antes de realizar esta operación, es esencial llevar a cabo la vectorización de los chunks.

Para este propósito, he empleado el modelo multilingüe proporcionado por SentenceTransformers, una técnica que transforma las oraciones en representaciones vectoriales.

Este proceso de vectorización es fundamental para representar el contenido de manera numérica y permitir su almacenamiento y manipulación en la base de datos vectorial.

In [ ]:
modelo_embeddings = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')


A continuación, he obtenido los embeddings para cada fragmento de la variable 'chunks_limpios'. A modo de ejemplo, presento los embeddings correspondientes al primer fragmento.

In [ ]:
# Obtengo embeddings de cada chunk
embeddings_chunks = [modelo_embeddings.encode(text) for text in chunks_limpios]

# Ejemplo de los embeddings del primer chunk
print("Embeddings del primer chunk:")
print(embeddings_chunks[0])

Embeddings del primer chunk:
[-3.27446125e-02  3.36861648e-02 -1.52938291e-02  2.42435634e-02
  1.03460751e-01  2.03105900e-02  4.94881533e-02 -1.77924410e-02
  8.32786337e-02  2.72743590e-03  8.82556289e-03  3.37455980e-02
  8.35383087e-02 -8.79838914e-02 -4.90695285e-03 -2.47511789e-01
 -5.45707112e-03  3.74743827e-02 -2.04594471e-02  4.02803794e-02
  7.54378783e-03  6.30736798e-02  1.26260027e-01  4.96611074e-02
  2.68890634e-02 -2.06436682e-02  1.02144247e-02  1.56673789e-02
  4.60897423e-02  5.35539538e-02  3.88960615e-02  5.83274253e-02
  1.09988600e-02 -3.31395715e-02  1.97323598e-03 -4.40786444e-02
 -1.52898235e-02 -3.38912942e-02  5.92442267e-02 -7.69888237e-02
  9.25239772e-02 -1.02634959e-01  9.43664741e-03  1.40964212e-02
 -1.59559950e-01  2.24269163e-02 -2.41304049e-03 -4.07381691e-02
 -2.22409572e-02 -9.84585565e-03  3.33989598e-02  5.76558337e-02
 -1.35075584e-01 -5.39415665e-02 -1.43583156e-02  2.21183207e-02
 -7.61065558e-02 -6.69264197e-02  1.61894962e-01  5.83055057e

En el siguiente fragmento de código, utilizo el cliente Chroma para crear una nueva colección llamada "superheroes". También genero identificadores únicos (IDs) para cada fragmento en la variable 'chunks_limpios'.

Luego, convierto los embeddings de NumPy en listas para asegurar que cumplan con los requisitos de la función add() de la colección Chroma.

Finalmente, añado los documentos a la colección, donde cada documento está representado por su embedding, el fragmento de texto correspondiente y el ID único asociado.

In [ ]:
# Cliente Chroma
chroma_client = chromadb.Client()

# Creo la colección
collection = chroma_client.create_collection(name="superheroes")

# Creo IDs
ids = [f'id{i+1}' for i in range(len(chunks_limpios))]

embeddings_chunks_listas = [embedding.tolist() for embedding in embeddings_chunks]

# Añado documentos a la colección
collection.add(
    embeddings=embeddings_chunks_listas,
    documents=chunks_limpios,
    ids=ids
)

Finalmente, veamos cómo se realiza una consulta a la base de datos vectorial, de forma similar a como hará el modelo de NLP que desarrollaré más adelante.

Aquí, se realiza una consulta a la colección "superheroes" utilizando un vector de embedding generado a partir de la pregunta "¿Cómo es la vestimenta de los superhéroes?". Primero, transformo el embedding de NumPy a una lista para que sea compatible con la función query() de la colección Chroma.

La consulta se ejecuta con la función query(), especificando el vector de embedding de la pregunta y solicitando hasta 5 resultados (n_results=5). Luego, itero sobre los resultados obtenidos e imprimo cada documento asociado al embedding, mostrando así la información recuperada para responder a la pregunta.

In [ ]:
pregunta = "¿Cómo es la vestimenta de los superhéroes?"
pregunta_embedding = modelo_embeddings.encode(pregunta).tolist()

results = collection.query(query_embeddings=[pregunta_embedding], n_results=5)
for result in results['documents'][0]:
    print(result)
    print('\n')

 21 10. superheroes fashion and fantasy septiembre 2008 inaugura exposicion superheroes fashion and fantasy metropolitan museum of art new york analiza pone relieve relacion moda universo superheroes . muestra exploraba asociaciones simbolicas metaforicas moda superheroe . trajes usados peliculas alta costura vanguardia ropa deportiva alto rendimiento muestra revelaba superheroe podia llegar ser metafora moda igual capacidades superheroes podia transformar cuerpo humano


 respecto figura superheroe blindado the armored body determinar primero falta superpoderes acompanados arsenal gadgets lucha crimen . trajes gadgets inspirados caso batman murcielago cuya capa anunciada dibujos ornithopter leonardo da vinci . modus operandi queda reflejado color diseno asociaciones noche miedo sobrenatural ocultacion sigilo sorpresa . caso ironman sirve metafora eficaz fusion cuerpo tecnologia


 j. c. hombres mujeres llevaron tunica llamada chiton simple rectangulo tela colgaba cuerpo ofrecia multip

## Base de datos de grafos

Como segunda fuente de información para el chatbot, utilicé la base de datos de grafos online de Wikidata. En situaciones en las que el usuario realice consultas específicas sobre un superhéroe en particular, el modelo se encargará de gestionar y realizar consultas directas a la base de datos de grafos de Wikidata.

En el siguiente bloque de código, definí la función 'contexto_grafos' que realiza una consulta a la base de datos de Wikidata para obtener información contextual sobre un superhéroe específico. El mismo se identifica a través de su etiqueta en inglés proporcionada como parámetro de entrada (superheroe). La función inicia una sesión en Wikidata y ejecuta una consulta SPARQL para recuperar el ID único del superhéroe.

Una vez obtenido el ID, se realizan consultas para obtener información detallada sobre propiedades específicas relacionadas con el superhéroe, como datos como nombre completo, género, lugar de nacimiento, fecha de nacimiento, nacionalidad, entre otros.

El resultado de estas consultas se organiza en un formato que luego se retorna como un string. En caso de cualquier error durante el proceso, ya sea en la conexión a Wikidata o en las consultas SPARQL, se proporcionan mensajes informativos de error para facilitar la identificación y resolución de problemas.

Cabe destacar que la función la elaboré con ayuda de un repositorio de Git Hub sobre el tema ya que durante el cursado de la materia no hemos visto suficiente código sobre la extracción de datos para una base de datos online.

In [ ]:
# Aquí se debe escribir un usuario y contraseña de Wikidata

usuario_wikidata = 'Abrilr1604'
contraseña_wikidata = ''

Defino dentro de 'propiedades' los códigos más relevantes que coinciden con información acerca de un superhéroe en cuestión.

In [ ]:
propiedades = ['P21', # sexo/género
               'P27', # país
               'P1477', # nombre de nacimiento
               'P19', # ciudad de nacimiento
               'P22', # padre
               'P25', # madre
               'P26', # esposo/esposa
               'P40', # hijos
               'P1038', # familiares
               'P106', # ocupación
               'P69', # educación
               'P1884', # color de pelo
               'P463', # miembro de (ej: Avengers)
               'P170', # creador
               'P175', # actores q lo interpretaron
               'P2048', # altura
               'P7047' # enemigos
               ]

In [ ]:
def contexto_grafos(superheroe):
    '''Recibe el nombre de un superhéroe y, a través de consultas a Wikidata, devuelve
    información de contexto relacionada a dicho personaje.'''

    sesion = wdi_login.WDLogin(usuario_wikidata, contraseña_wikidata)

    # Consulta SPARQL
    consulta_sparql = f'''
    SELECT ?sujeto ?sujetoLabel
    WHERE {{
      ?sujeto rdfs:label "{superheroe}"@en.
      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }}
    }}
    '''

    result = wdi_core.WDItemEngine.execute_sparql_query(consulta_sparql, as_dataframe=True)

    # Obtengo el ID del superhéroe
    try:
        valor = result['sujeto'].str.split('/').str[-1]
    except (KeyError, AttributeError, IndexError):
        return ''

    # Obtengo el primer resultado
    superheroe_ID = valor.values[0]

    contexto = f'{superheroe}  :\n'

    for propiedad_id in propiedades:
        # Construir la consulta SPARQL para obtener los valores por sujeto y propiedad
        consulta_sparql = f"""
        SELECT ?property ?propertyLabel ?value ?valueLabel
        WHERE {{
          wd:{superheroe_ID} wdt:{propiedad_id} ?value.
          ?property wikibase:directClaim wdt:{propiedad_id}.
          SERVICE wikibase:label {{ bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }}
        }}
        """

        endpoint_url = "https://query.wikidata.org/sparql"

        # Encabezados HTTP para la consulta
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0',
            'Accept': 'application/json'
        }

        params = {
            'query': consulta_sparql,
            'format': 'json'
        }

        # Solicitud HTTP GET
        respuesta = requests.get(endpoint_url, headers=headers, params=params)

        # Si la solicitud fue exitosa --> convierto la respuesta a JSON
        if respuesta.status_code == 200:
            resultados = respuesta.json()
            valores = [(item['propertyLabel']['value'], item['valueLabel']['value']) for item in resultados['results']['bindings']]
            # Agrego los valores de cada propiedad al contexto
            for propiedad, valor in valores:
                contexto += f'{propiedad}: {valor} \n'
        else:
            contexto = '' # sino --> cadena vacía

    return contexto

Finalmente, veamos un ejemplo del contexto que brinda la función si le pasamos como superhéroe a 'Spider-Man'.

In [ ]:
print(contexto_grafos("Spider-Man"))

Spider-Man  :
sex or gender: male 
country of citizenship: United States of America 
birth name: Peter Benjamin Parker 
place of birth: Queens 
father: Richard Parker 
mother: Mary Parker 
spouse: Mary Jane Watson 
child: Spider-Girl 
child: Benjamin Richard Parker 
child: Felicity Hardy 
relative: Uncle Ben 
relative: Aunt May 
relative: Q118951942 
occupation: scientist 
occupation: photographer 
occupation: teacher 
occupation: superhero 
occupation: inventor 
educated at: Empire State University 
educated at: Midtown High School 
hair color: black hair 
member of: Avengers 
creator: Stan Lee 
creator: Steve Ditko 
performer: Tom Holland 
height: 178 
enemy: Lizard 
enemy: Tombstone 
enemy: Wraith 
enemy: Speed Demon 
enemy: Riot 
enemy: Chameleon 
enemy: Rhino 
enemy: Doctor Octopus 
enemy: Burglar 
enemy: Alyosha Kravinoff 
enemy: Kraven the Hunter 
enemy: Green Goblin 
enemy: Hobgoblin 
enemy: Sandman 
enemy: Morbius, the Living Vampire 
enemy: Beetle 
enemy: Venom 
enemy: Kingpi

Es importante destacar que la correcta escritura del nombre del superhéroe es necesaria. En caso de errores ortográficos o imprecisiones, Wikidata podría no ser capaz de identificar el ID correspondiente o, incluso, recuperar uno con datos incorrectos o incompletos. En tales situaciones, la función asociada devolverá una cadena vacía

## Datos tabulares

La última fuente de información para el chatbot se basa en datos tabulares almacenados en un dataframe. Este dataframe contiene información numérica relevante, como la fecha de lanzamiento, la duración en horas y minutos, y los ingresos brutos, relacionada con las adaptaciones cinematográficas más destacadas de superhéroes.

In [ ]:
data = pd.read_csv("mcu_films.csv")
data

,movie,length_hrs,length_min,release_date,opening_weekend_us,gross_us,gross_world
0,Iron Man,2,6,5/2/2008,98618668,319034126,585796247
1,The Incredible Hulk,1,52,6/12/2008,55414050,134806913,264770996
2,Iron Man 2,2,4,5/7/2010,128122480,312433331,623933331
3,Thor,1,55,5/6/2011,65723338,181030624,449326618
4,Captain America: The First Avenger,2,4,7/22/2011,65058524,176654505,370569774
5,Marvel's The Avengers,2,23,5/4/2012,207438708,623357910,1518815515
6,Iron Man 3,2,10,5/3/2013,174144585,409013994,1214811252
7,Thor: The Dark World,1,52,11/8/2013,85737841,206362140,644783140
8,Captain America: The Winder Soldier,2,16,4/4/2014,95023721,259766572,714421503
9,Guardians of the Galaxy,2,1,8/1/2014,94320883,333718600,773341024


Antes de definir una función para captar el contexto específico de una película, realizaré algunas modificaciones al dataframe para mejorar la comprensión de su contenido.

En primer lugar, añadiré una nueva columna llamada 'duracion_min' que representará la duración total de la película en minutos, en sustitución de las columnas actuales 'length_hrs' y 'length_min'.

In [ ]:
# Genero una nueva columna
data['duracion_min'] = data['length_hrs'] * 60 + data['length_min']

# Elimino las columnas 'length_hrs' y 'length_min'
data = data.drop(['length_hrs', 'length_min'], axis=1)

Luego, cambiaré el formato de la fecha de lanzamiento a "día/mes/año", ya que este formato es más comúnmente utilizado y reconocido en español.

In [ ]:
data['release_date'] = pd.to_datetime(data['release_date'], format='%m/%d/%Y').dt.strftime('%d/%m/%Y')

Finalmente, realizaré un cambio en los nombres de las variables para adaptarlos al idioma español.

In [ ]:
data = data.rename(columns={
    'movie': 'pelicula',
    'release_date': 'fecha_lanzamiento',
    'opening_weekend_us': 'estreno_fin_semana_us',
    'gross_us': 'ingresos_us',
    'gross_world': 'ingresos_mundiales'
})

data

,pelicula,fecha_lanzamiento,estreno_fin_semana_us,ingresos_us,ingresos_mundiales,duracion_min
0,Iron Man,02/05/2008,98618668,319034126,585796247,126
1,The Incredible Hulk,12/06/2008,55414050,134806913,264770996,112
2,Iron Man 2,07/05/2010,128122480,312433331,623933331,124
3,Thor,06/05/2011,65723338,181030624,449326618,115
4,Captain America: The First Avenger,22/07/2011,65058524,176654505,370569774,124
5,Marvel's The Avengers,04/05/2012,207438708,623357910,1518815515,143
6,Iron Man 3,03/05/2013,174144585,409013994,1214811252,130
7,Thor: The Dark World,08/11/2013,85737841,206362140,644783140,112
8,Captain America: The Winder Soldier,04/04/2014,95023721,259766572,714421503,136
9,Guardians of the Galaxy,01/08/2014,94320883,333718600,773341024,121



Cuando el usuario formula preguntas específicas sobre estos temas, el LLM (Lenguaje de Modelado de Lenguaje) está diseñado para invocar la función denominada 'contexto_pelicula'. Esta función, a su vez, proporciona contextos informativos específicos relacionados con una película particular que se encuentra dentro del dataframe. Este enfoque amplía la capacidad del chatbot para responder con información detallada y específica sobre adaptaciones cinematográficas de superhéroes, aprovechando la riqueza de datos contenidos en el dataframe.

In [ ]:
def contexto_pelicula(nombre_pelicula):

    '''Recibe el nombre de una película y, si pertenece al dataframe, devuelve
    información relevante sobre la misma.'''

    result = ""
    pelicula_encontrada = False

    for index, row in data.iterrows():
        if row['pelicula'].lower() == nombre_pelicula.lower():
            pelicula_encontrada = True
            result += f"Película: {row['pelicula']}\n"
            result += f"Fecha de lanzamiento: {row['fecha_lanzamiento']}\n"
            result += f"Ingresos fin de semana apertura US: {row['estreno_fin_semana_us']}\n"
            result += f"Ingresos US: {row['ingresos_us']}\n"
            result += f"Ingresos mundiales: {row['ingresos_mundiales']}\n"
            result += f"Duración: {row['duracion_min']} minutos\n"
            result += "\n"
            break  # Terminar el bucle al encontrar la película

    if not pelicula_encontrada:
        result = f"No se encontró información para la película: {nombre_pelicula}"

    return result

Veamos un ejemplo si queremos obtener el contexto para la película 'Iron Man'.

In [ ]:
print(contexto_pelicula('Iron Man'))

Película: Iron Man
Fecha de lanzamiento: 02/05/2008
Ingresos fin de semana apertura US: 98618668
Ingresos US: 319034126
Ingresos mundiales: 585796247
Duración: 126 minutos




## LLM

Una vez constituidas las fuentes de conocimiento para el chatbot, el siguiente paso es el armado del modelo. Para ello implementé una función llamada 'zephyr_chat_template' y otra llamada 'clasificar_pregunta'.

La función 'zephyr_chat_template utiliza' la plantilla Jinja para formatear mensajes en un formato específico. Toma una lista de mensajes, cada uno con un rol ("system", "user", "assistant", u otro) y su contenido, y utiliza la plantilla para generar un formato coherente. Además, tiene un parámetro opcional add_generation_prompt que indica si se debe agregar un prompt de generación al final.

Para ejecutar el código y poner en funcionamiento el modelo, será necesario una clave de Hugging Face, la cual deberá ser almacenada dentro de la variable 'clave_huggingface'

In [ ]:
clave_huggingface= ''

La variable messages: List[Dict[str, str]] es una lista de diccionarios en Python. Cada diccionario en la lista representa un mensaje en una conversación. Los mensajes están estructurados con roles ("system", "user" o "assistant") y contenido.

In [ ]:
def zephyr_chat_template(messages, add_generation_prompt=True):
    '''Recibe un mensaje y define una plantilla Jinja para formatearlo
    en un formato específico'''

    template_str  = "{% for message in messages %}"
    template_str += "{% if message['role'] == 'user' %}"
    template_str += "<|user|>{{ message['content'] }}\n"
    template_str += "{% elif message['role'] == 'assistant' %}"
    template_str += "<|assistant|>{{ message['content'] }}\n"
    template_str += "{% elif message['role'] == 'system' %}"
    template_str += "<|system|>{{ message['content'] }}\n"
    template_str += "{% else %}"
    template_str += "<|unknown|>{{ message['content'] }}\n"
    template_str += "{% endif %}"
    template_str += "{% endfor %}"
    template_str += "{% if add_generation_prompt %}"
    template_str += "<|assistant|>\n"
    template_str += "{% endif %}"

    # Creo un objeto de plantilla con la cadena de plantilla
    template = Template(template_str)

    # Renderizo la plantilla con los mensajes proporcionados
    return template.render(messages=messages, add_generation_prompt=add_generation_prompt)

La segunda función que definí es 'clasificar_pregunta'. La misma simula una conversación entre un usuario y un asistente utilizando la plantilla generada por zephyr_chat_template. Luego, hace una solicitud a la API de Hugging Face para generar texto basado en el prompt de la conversación. El objetivo de esta función es clasificar la pregunta recibida por parámetro en 3 opciones: 'Historia', 'Superhéroe' o 'Película'. Esta clasificación es crucial, ya que determina a qué fuente de información deberá consultar el LLM posteriormente, dependiendo de la naturaleza de la pregunta.


El modelo que decidí utilizar es zephyr-7b-beta de Hugging Face, el cual es el segundo modelo de la serie Zephyr y está entrenado para actuar como asistente útile.

In [ ]:
def clasificar_pregunta(prompt: str, clave_huggingface, max_new_tokens: int = 768) -> None:
    messages: List[Dict[str, str]] = [
                                {
                                  "role": "system",
                                  "content": "Eres un asistente bibliográfico, especializado en identificar el tema u objeto referido en una pregunta o tarea."
                                },
                                {
                                  "role": "user",
                                  "content": "¿Cuál es el nombre completo de Captain America?"
                                },
                                {
                                  "role": "assistant",
                                  "content": "Superhéroe: [Captain America]"
                                },
                                {
                                  "role": "user",
                                  "content": "¿Quiénes son los enemigos de Hulk?"
                                },
                                {
                                  "role": "assistant",
                                  "content": "Superhéroe: [Hulk]"
                                },
                                {
                                  "role": "user",
                                  "content": "¿Quién es la madre de Spider-Man?"
                                },
                                {
                                  "role": "assistant",
                                  "content": "Superhéroe: [Spider-Man]"
                                },
                                {
                                  "role": "user",
                                  "content": "¿En qué fecha fue lanzada la película Thor: Ragnarok?"
                                },
                                {
                                  "role": "assistant",
                                  "content": "Película: [Thor: Ragnarok]"
                                },
                                {
                                  "role": "user",
                                  "content": "¿Cuánto dura la película Black Panther?"
                                },
                                {
                                  "role": "assistant",
                                  "content": "Película: [Black Panther]"
                                },
                                {
                                  "role": "user",
                                  "content": "¿Cuáles son las características de los superhéroes?"
                                },
                                {
                                  "role": "assistant",
                                  "content": "Historia"
                                },
                                {
                                  "role": "user",
                                  "content": "¿Cuál es el rol de las mujeres superhéroes?"
                                },
                                {
                                  "role": "assistant",
                                  "content": "Historia"
                                },
                                {
                                  "role": "user",
                                  "content": f'Responde con sólo una de las siguientes formas o palabras. No utilices conocimiento previo. No agregues información:\n\
                                  "Superhéroe: [Nombre del superhéroe mencionado en la pregunta]" si es una pregunta sobre un superhéroe en específico. \n\
                                  "Película: [Nombre de la película mencionada en la pregunta]" si es una pregunta sobre las películas de superhéroes.\n\
                                  "Historia" si la pregunta es sobre alguna definición o característica sobre la literatura de superhéroes en general.\n\
                                  El nombre del superhéroe o de la película debes escribirlo entre corchetes.\n\
                                  -------------------------------------------\n\
                                  La pregunta es la siguiente: \n\
                                  {prompt}'
                                }
                              ]


    try:
        prompt_formatted: str = zephyr_chat_template(messages, add_generation_prompt=True)

        # URL de la API de Hugging Face para la generación de texto
        api_url = "https://api-inference.huggingface.co/models/HuggingFaceH4/zephyr-7b-beta"

        # Cabeceras para la solicitud
        headers = {"Authorization": f"Bearer {clave_huggingface}"}

        # Datos para enviar en la solicitud POST
        data = {
            "inputs": prompt_formatted,
            "parameters": {
                "max_new_tokens": max_new_tokens,
                "temperature": 0.7,
                "top_k": 50,
                "top_p": 0.95
            }
        }

        # Realizo la solicitud POST
        response = requests.post(api_url, headers=headers, json=data)

        # Extraigo respuesta
        respuesta = response.json()[0]["generated_text"][len(prompt_formatted):]
        print(type(respuesta))
        return respuesta

    except Exception as e:
        print(f"An error occurred: {e}")

A continuación, se muestran 3 ejemplos, uno para cada "categoría" con el fin de comprobar que el modelo clasifica bien las preguntas.

In [ ]:
# Pregunta sobre historia/definiciones (BBDD vectorial)

pregunta_1 = '¿Cuándo fue el inico de los cómics?'
respuesta_1 = clasificar_pregunta(pregunta_1, clave_huggingface=clave_huggingface)
print(respuesta_1)

<class 'str'>
Historia: "La pregunta es la siguiente: ¿Cuándo fue el inicio de los cómics?" No es necesario agregar "Historia" en este caso, pero puedes utilizar esa etiqueta si quieres clarificar que la pregunta se refiere a la historia de la literatura de superhéroes en general. En este caso, no es necesario agregar el nombre de un superhéroe o de una película, por lo que no se necesita ponerlos entre corchetes.


Al realizar preguntas relacionadas con la historia o definiciones en el ámbito de la literatura de superhéroes, observamos que el modelo demuestra capacidad para clasificar adecuadamente estas consultas, aunque brinda información de más, aún cuando le pedí al modelo que no lo haga. A pesar de que en este ejemplo solo incluí una pregunta concreta, he realizado varias pruebas similares dentro de la misma categoría, y el modelo ha logrado clasificar correctamente todas ellas.

A continuación, llevé a cabo una evaluación similar para preguntas específicas sobre superhéroes y sus adaptaciones cinematográficas, y el modelo demostró nuevamente capacidad para realizar clasificaciones precisas.

Cabe aclarar que, durante las pruebas, confirmé que la solicitud al modelo de incluir el nombre del superhéroe y la película entre corchetes me permitió obtener el mismo en un formato que más adelante me facilitó la búsqueda en Wikidata y en el Dataframe. A pesar de que exploré otras opciones como %, & y $, la inclusión de corchetes resultó ser la única forma efectiva de garantizar la obtención del formato necesario para evitar complicaciones futuras.

In [ ]:
# Pregunta sobre algún superhéroe (BBDD de grafos)

pregunta_2 = '¿Cuál es el nombre completo de Spider-Man?'
respuesta_2 = clasificar_pregunta(pregunta_2, clave_huggingface=clave_huggingface)
print(respuesta_2)

<class 'str'>
Superhéroe: [Spider-Man]


In [ ]:
# Pregunta sobre cine de superhéroes (Datos tabulares)

pregunta_3 = '¿Cuánto dura la película Captain America: The First Avenger?'
respuesta_3 = clasificar_pregunta(pregunta_3, clave_huggingface=clave_huggingface)
print(respuesta_3)

<class 'str'>
Película: [Captain America: The First Avenger]


Ahora que el modelo es capaz de clasificar correctamente las preguntas en sólo una o dos palabras, he desarrollado la función 'obtener_contexto', diseñada para tomar decisiones basadas en dichas clasificaciones.

Cuando la pregunta se refiere a temas de historia o definiciones en el ámbito de la literatura de superhéroes, la función busca en la base de datos vectorial los cinco fragmentos ('chunks') con mayor similitud y los devuelve como contexto en formato de cadena.

En el caso de preguntas que incluyen un superhéroe específico (identificado entre corchetes, de allí la elección de formato para facilitar la búsqueda), la función extrae el nombre del superhéroe y llama a la función 'contexto_grafos', que consulta la base de datos de grafos online y devuelve información relevante sobre ese superhéroe como contexto.

Finalmente, cuando la pregunta se centra en adaptaciones cinematográficas de superhéroes (películas), la función invoca a 'contexto_pelicula', que genera un contexto en formato de cadena conteniendo información sobre la película mencionada.

In [ ]:
def obtener_contexto(respuesta, pregunta_usuario = None):
    # Divido la respuesta en palabras
    palabras = respuesta.split()

    # Si la pregunta es sobre historia/definiciones --> BBDD ChromaDB
    if palabras and palabras[0] == "Historia:":
        contexto = ''
        chroma_client = chromadb.Client()
        collection = chroma_client.get_collection(name="superheroes")
        query_embedding = modelo_embeddings.encode(pregunta_usuario).tolist() # vectorizo pregunta del usuario
        results = collection.query(query_embeddings= [query_embedding], n_results=5) # obtengo primeros 5 chunks
        for result in results['documents'][0]:
          contexto += result

        return (contexto, "Base de datos vectorial")

    # Si la pregunta es sobre un superhéroe específico --> BBDD de Grafos
    elif palabras and palabras[0] == "Superhéroe:":
        # Busco el nombre del superhéroe entre corchetes
        indice_corchete_inicio = respuesta.find("[")
        indice_corchete_fin = respuesta.find("]", indice_corchete_inicio)

        if indice_corchete_inicio != -1 and indice_corchete_fin != -1:
            nombre_superheroe = respuesta[indice_corchete_inicio + 1:indice_corchete_fin]
            contexto_personaje = contexto_grafos(nombre_superheroe)
            return (contexto_personaje, 'Base de datos de grafos')

        else:
            return "Superhéroe", None

    # Si la pregunta es sobre una película --> Datos Tabulares (Dataframe)
    elif palabras and palabras[0] == "Película:":
        # Busco el nombre de la película entre corchetes
        indice_corchete_inicio = respuesta.find("[")
        indice_corchete_fin = respuesta.find("]", indice_corchete_inicio)

        if indice_corchete_inicio != -1 and indice_corchete_fin != -1:
            nombre_pelicula = respuesta[indice_corchete_inicio + 1:indice_corchete_fin]
            contexto_pelicula_elegida = contexto_pelicula(nombre_pelicula)
            return (contexto_pelicula_elegida, 'Datos Tabulares')

    # Si la respuesta no tiene el formato esperado
    else:
        return "Categoría no identificada", None

A continuación, pruebo que la función devuelva los resultados esperados.

In [ ]:
# Pregunta sobre historia/definiciones (BBDD vectorial)

contexto_1, fuente_1 = obtener_contexto(respuesta_1, pregunta_1)

print("Contexto: ", contexto_1, "\n")
print("Fuente elegida: ", fuente_1)

Contexto:   hecho 1961 editorialque hacia tiempo publicaba historietas superheroes eraconocida comics monstruos . finales delmismo ano surgieron 4 fantasticos 1961 gracias exitodieron paso conocidos superheroes grupos habiapasado llamarse marvel comics hulk 1962 spiderman 1962 ironman 1963 dr. strange 1963 the avengers 1963 xmen 1963.los 4 fantasticos the fantastic fourlos 4 fantasticos 1961 supuso recuperacion historias desuperheroes parte marvel jerry siegel confeso idea basica superman habia ocurrido hacia 1932 ano despues puso desarrollarla amigo dibujante joe shuster personaje distinto conocemos . primeras vinetas llevaba leotardos azules calzones rojos caracteristica s torax . superman epoca iba camiseta parecia mas bien forzudo circo aquellos hercules pululaban espectaculos feria . personaje enlazaba tradicion heroes mitologicos aquiles goliat sanson . aunque paginas poca gente sabe proyecto rechazado segun abraham garcia3estos dos grupos confluyeron tipo heroe oculta suidentida

In [ ]:
# Pregunta sobre un superhéroe específico (BBDD de Grafos)

contexto_2, fuente_2 = obtener_contexto(respuesta_2, pregunta_2)

print("Contexto: ", contexto_2, "\n")
print("Fuente elegida: ", fuente_2)

Contexto:  Spider-Man  :
sex or gender: male 
country of citizenship: United States of America 
birth name: Peter Benjamin Parker 
place of birth: Queens 
father: Richard Parker 
mother: Mary Parker 
spouse: Mary Jane Watson 
child: Spider-Girl 
child: Benjamin Richard Parker 
child: Felicity Hardy 
relative: Uncle Ben 
relative: Aunt May 
relative: Q118951942 
occupation: scientist 
occupation: photographer 
occupation: teacher 
occupation: superhero 
occupation: inventor 
educated at: Empire State University 
educated at: Midtown High School 
hair color: black hair 
member of: Avengers 
creator: Stan Lee 
creator: Steve Ditko 
performer: Tom Holland 
height: 178 
enemy: Lizard 
enemy: Tombstone 
enemy: Wraith 
enemy: Speed Demon 
enemy: Riot 
enemy: Chameleon 
enemy: Rhino 
enemy: Doctor Octopus 
enemy: Burglar 
enemy: Alyosha Kravinoff 
enemy: Kraven the Hunter 
enemy: Green Goblin 
enemy: Hobgoblin 
enemy: Sandman 
enemy: Morbius, the Living Vampire 
enemy: Beetle 
enemy: Venom 
en

In [ ]:
# Pregunta sobre una película (Datos Tabulares)

contexto_3, fuente_3 = obtener_contexto(respuesta_3, pregunta_3)

print("Contexto: ", contexto_3, "\n")
print("Fuente elegida: ", fuente_3)

Contexto:  Película: Captain America: The First Avenger
Fecha de lanzamiento: 22/07/2011
Ingresos fin de semana apertura US: 65058524
Ingresos US: 176654505
Ingresos mundiales: 370569774
Duración: 124 minutos

 

Fuente elegida:  Datos Tabulares


Finalmente, definí la funcion 'chatbot', la cual recibe un prompt y un contexto.

Dicha función simula una interacción entre un usuario y un asistente. Se utiliza la plantilla zephyr_chat_template para estructurar la conversación. La información de contexto se proporciona al asistente, y luego se plantea una pregunta específica al modelo (la ingresada por el usuario). La función realiza una solicitud a la API de Hugging Face para generar una respuesta coherente y basada en hechos en función del contexto y la pregunta proporcionados.

La URL de la API de Hugging Face, las cabeceras y los datos para la solicitud POST se configuran adecuadamente. Después de realizar la solicitud, se extrae y devuelve la respuesta generada por el modelo, considerando la información de contexto y la pregunta proporcionada.

En caso de que ocurra algún error durante el proceso, se maneja y se imprime un mensaje indicando que se produjo un error.  

In [ ]:
def chatbot(prompt: str, context: str, clave_huggingface, max_new_tokens: int = 768) -> None:
    '''Recibe una pregunta (prompt) y un contexto y devuelve una respuesta en base a
    ambos'''
    messages: List[Dict[str, str]] = [
                                    {
                                        "role": "system",
                                        "content": "Eres un asistente útil que siempre responde con respuestas veraces, útiles y basadas en hechos."
                                    },
                                    {
                                        "role": "user",
                                        "content": f"La información de contexto es la siguiente:\n\
                                        ---------------------\n\
                                        {context}\n\
                                        ---------------------\n\
                                        Dada la información de contexto anterior, y sin utilizar conocimiento previo, responde la siguiente pregunta.\n\
                                        Pregunta: {prompt}\n\
                                        Respuesta: "
                                    }
                                ]

    try:
        prompt_formatted: str = zephyr_chat_template(messages, add_generation_prompt=True)

        # URL de la API de Hugging Face para la generación de texto
        api_url = "https://api-inference.huggingface.co/models/HuggingFaceH4/zephyr-7b-beta"

        # Cabeceras para la solicitud
        headers = {"Authorization": f"Bearer {clave_huggingface}"}

        # Datos para enviar en la solicitud POST
				# Sobre los parámetros: https://huggingface.co/docs/transformers/main_classes/text_generation
        data = {
            "inputs": prompt_formatted,
            "parameters": {
                "max_new_tokens": max_new_tokens,
                "temperature": 0.7,
                "top_k": 50,
                "top_p": 0.95
            }
        }

        # Realizo la solicitud POST
        response = requests.post(api_url, headers=headers, json=data)

        # Extraigo respuesta
        respuesta = response.json()[0]["generated_text"][len(prompt_formatted):]
        return respuesta

    except Exception as e:
        print(f"An error occurred: {e}")


## Uso del LLM

Probemos, finalmente, el funcionamiento del chatbot realizándole las mismas preguntas de antes.

In [ ]:
print(f'Pregunta: {pregunta_1}\n')
print(f'Respuesta:\n {chatbot(pregunta_1, contexto_1, clave_huggingface=clave_huggingface)}')

Pregunta: ¿Cuándo fue el inico de los cómics?

Respuesta:
 La pregunta se refiere a los cómics como formato de historietas, en lugar de la industria de cómics como un todo. Según la información de contexto, la primera historieta con un personaje superhéroe, Superman, fue publicada en el número 1 de la revista Action Comics de DC Comics en junio de 1938. Así que, en esencia, se puede decir que el inicio de los cómics en su forma actual, con historias protagonizadas por superhéroes, fue en 1938. Sin embargo, la primera editorial que publicaba historietas, National Allied Publications, se fundó en 1934.


In [ ]:
print(f'Pregunta: {pregunta_2}\n')
print(f'Respuesta:\n {chatbot(pregunta_2, contexto_2, clave_huggingface=clave_huggingface)}')

Pregunta: ¿Cuál es el nombre completo de Spider-Man?

Respuesta:
 El nombre completo de Spider-Man según la información de contexto proporcionada es Peter Benjamin Parker.


In [ ]:
print(f'Pregunta: {pregunta_3}\n')
print(f'Respuesta:\n {chatbot(pregunta_3, contexto_3, clave_huggingface=clave_huggingface)}')

Pregunta: ¿Cuánto dura la película Captain America: The First Avenger?

Respuesta:
 La duración de la película Captain America: The First Avenger es de 124 minutos, según la información de contexto proporcionada.


A continuación, un bloque de código para que interactuar con el chatbot.

In [ ]:
def hacer_pregunta_interactiva():
    while True:
        # Solicito al usuario que ingrese una pregunta
        print("Hola, soy un chatbot especializado en superhéroes. ¡Hazme una pregunta! \n")
        pregunta_usuario = input("Ingresa tu pregunta o escribe 'salir' para salir: ")

        # Verifico si el usuario desea salir
        if pregunta_usuario.lower() == 'salir':
            print("¡Hasta luego!")
            break

        # Clasifico la pregunta
        respuesta = clasificar_pregunta(pregunta_usuario, clave_huggingface=clave_huggingface)

        # Obtengo contexto según el tipo de pregunta
        contexto_usuario, fuente_usuario = obtener_contexto(respuesta, pregunta_usuario)

        # Llamo a la función chatbot con la pregunta y el contexto proporcionados
        respuesta_chatbot = chatbot(pregunta_usuario, contexto_usuario, clave_huggingface=clave_huggingface)

        # Muestro la pregunta y la respuesta generada por el chatbot
        print(f'\nPregunta: {pregunta_usuario}\n')
        print(f'Respuesta:\n {respuesta_chatbot}\n')

# Llamo a la función para comenzar la interacción
hacer_pregunta_interactiva()

Hola, soy un chatbot especializado en superhéroes. ¡Hazme una pregunta! 

Ingresa tu pregunta o escribe 'salir' para salir: ¿Cómo es la vestimenta de los superhéroes?
<class 'str'>

Pregunta: ¿Cómo es la vestimenta de los superhéroes?

Respuesta:
 La vestimenta de los superhéroes es variada y refleja sus poderes, capacidades y personalidad. En algunos casos, se inspiran en trajes de alta costura o ropa deportiva de alto rendimiento, mientras que en otras ocasiones, se incorporan gadgets y elementos tecnológicos. El color y el diseño de la ropa reflejan sus poderes y habilidades, y pueden tener asociaciones simbólicas y metaforicas. Los superhéroes llevan expresiones similares en sus cuerpos marcados, y los elementos de su traje hablan acerca de sus poderes y capacidades, incluyendo defensa, lucha y valores. En general, la vestimenta de los superhéroes es suficientemente flexible y puede soportar complicadas secuencias de acción sin distraer al público. Los superhéroes de la primera épo